# Building Populations and Factions

In [1]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import sys, os
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")


%load_ext lab_black

In [2]:
from app import creators

In [3]:
from app.creators import homeworld
from app.objects import species, population

Example data from the input form:

In [4]:
data = {
    "label": "form",
    "name": "worldgenform",
    "objid": "0000000000001",
    "owner": "user.username",
    "username": "user.username",
    "accountid": "0000000000001",
    "conformity": 0.1,
    "constitution": 0.5,
    "literacy": 0.5,
    "aggression": 0.5,
    "num_planets": 4,
    "num_moons": 10,
    "starting_pop": 30,
    "organics": 0.5,
    "minerals": 0.5,
}

In [5]:
nodes, edges = homeworld.build_people(data)

In [12]:
[n for n in nodes if n["label"] == "faction"]

[{'name': 'Rocziqawi',
  'objid': '4008773813677',
  'label': 'faction',
  'lat': -0.2,
  'long': -0.168,
  'pop_loactions': [[0, 0],
   [-2, -2],
   [2, -2],
   [-2, 0],
   [-1, -1],
   [0, 2],
   [-2, 2],
   [2, 0],
   [1, -3]]},
 {'name': 'Ler',
  'objid': '2095607860319',
  'label': 'faction',
  'lat': 0.057,
  'long': 0.006,
  'pop_loactions': [[0, 0],
   [-1, -1],
   [-2, 0],
   [0, 2],
   [-1, 1],
   [-1, 3],
   [0, -2],
   [1, 1],
   [-2, 2],
   [-2, -2],
   [1, -1],
   [2, 0]]},
 {'name': 'Ce',
  'objid': '8423670952485',
  'label': 'faction',
  'lat': -0.154,
  'long': 0.182,
  'pop_loactions': [[0, 0], [-2, -2], [-1, -1], [-2, 2], [-2, 0], [2, 0]]},
 {'name': 'Cor',
  'objid': '3346048701281',
  'label': 'faction',
  'lat': 0.296,
  'long': -0.02,
  'pop_loactions': [[0, 0],
   [-2, 2],
   [0, 2],
   [-2, 0],
   [1, -1],
   [2, -2],
   [-1, -1]]}]

In [7]:
0 in [[0, 0]]

False

In [8]:
options = [[1, 1], [-1, 1], [1, -1], [-1, -1]]
pick = options[np.random.choice([0, 1, 2, 3])]
pick

[1, 1]

In [9]:
new_pick = options[np.random.choice([0, 1, 2, 3])]
new_pick

[1, -1]

In [10]:
all_pick = np.add(pick, new_pick)
all_pick.tolist()

[2, 0]